In [19]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [20]:
## dataset :)
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [21]:
## split the data into training,validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [22]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8])

In [23]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [24]:
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [25]:
train_x.shape[1]

11

In [26]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Training the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [27]:
def objective(params):
    # MLflow tracking the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [28]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [36]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    
    # conducting the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # fetching the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001004 seconds
INFO:hyperopt.tpe:TPE using 0 trials


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 351ms/step - loss: 33.8972 - root_mean_squared_error: 5.8221
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.3181 - root_mean_squared_error: 2.1319 - val_loss: 0.7418 - val_root_mean_squared_error: 0.8613

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.0643 - root_mean_squared_error: 1.0317
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6510 - root_mean_squared_error: 0.8055 - val_loss: 0.6253 - val_root_mean_squared_error: 0.7908

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.7276 - root_mean_squared_error: 0.8530
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6048 - root_mean_squared_error: 0.7773 - val_loss: 0.5271 - val_root_mean_squared_error: 0.7260

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.4398 - root_mean_squared_error: 0.6632
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000000 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.726028


Epoch 1/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 12s 285ms/step - loss: 33.7592 - root_mean_squared_error: 5.8103
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.9977 - root_mean_squared_error: 5.5630 - val_loss: 20.0901 - val_root_mean_squared_error: 4.4822

Epoch 2/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 19.1354 - root_mean_squared_error: 4.3744
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.4790 - root_mean_squared_error: 4.1770 - val_loss: 11.3569 - val_root_mean_squared_error: 3.3700

Epoch 3/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 11.8572 - root_mean_squared_error: 3.4434
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7502 - root_mean_squared_error: 3.1197 - val_loss: 6.5167 - val_root_mean_squared_error: 2.5528

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 6

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001001 seconds
INFO:hyperopt.tpe:TPE using 2/2 trials with best loss 0.726028


Epoch 1/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 12s 282ms/step - loss: 29.1026 - root_mean_squared_error: 5.3947
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.1496 - root_mean_squared_error: 5.5809   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 31.1164 - root_mean_squared_error: 5.5779 - val_loss: 27.8895 - val_root_mean_squared_error: 5.2811

Epoch 2/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 27.3210 - root_mean_squared_error: 5.2270
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.3959 - root_mean_squared_error: 5.1374 - val_loss: 23.2859 - val_root_mean_squared_error: 4.8255

Epoch 3/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 23.8264 - root_mean_squared_error: 4.8812
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22.4793 - root_mean_squared_error: 4.7406 
46/46 ━━━━━━━

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001047 seconds
INFO:hyperopt.tpe:TPE using 3/3 trials with best loss 0.726028


Epoch 1/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 34s 777ms/step - loss: 36.4075 - root_mean_squared_error: 6.0339
19/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.2963 - root_mean_squared_error: 6.0244   
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.3859 - root_mean_squared_error: 5.9478
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 34.7013 - root_mean_squared_error: 5.8894 - val_loss: 28.6203 - val_root_mean_squared_error: 5.3498

Epoch 2/3                                                                    

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 30.2444 - root_mean_squared_error: 5.4995
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.9805 - root_mean_squared_error: 5.1935 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 26.5012 - root_mean_squared_error: 5.1468 - val_loss: 21.8024 - val_root_mean_squared_error: 4.6693

Epoch 3/3                                                                    

 1/46 ━━━━━━━

In [31]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("wine-quality")

2025/06/05 21:27:35 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/762296460380415000', creation_time=1749169655944, experiment_id='762296460380415000', last_update_time=1749169655944, lifecycle_stage='active', name='wine-quality', tags={}>

In [37]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

run_id = "b11a39636f2f44b9a199a09603e11d54"
model_uri = f"runs:/{run_id}/model"

# Registrar o modelo no repositório
model_name = "modelo_ANN_deep_learning"
client.create_registered_model(model_name)
client.create_model_version(name=model_name, source=model_uri, run_id=run_id)

print(f"Modelo registrado como: {model_name}")

2025/06/05 21:36:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_ANN_deep_learning, version 1


Modelo registrado como: modelo_ANN_deep_learning


In [ ]:
# import pandas as pd
# loaded_model.predict(pd.DataFrame(test_x))

NameError: name 'loaded_model' is not defined